# Using Pydantic with Phenopackets

[Pydantic](https://docs.pydantic.dev/latest/) is a popular and fast python library for working with object models in Python.

LinkML auto-generates pydantic. This repo contains pre-generated pydantic models for phenopackets
(in addition to an alternate python representation, using classic dataclasses).

In [7]:
import json

from phenopackets.pydantic.model import *

## Constructing a Phenopacket

Let's make a very basic phenopacket with a single phenotypic feature.

We'll do the feature first

In [14]:
pf1 = PhenotypicFeature(
    type=OntologyClass(id="HP:0030084", label="Clinodactyly"),
    onset=TimeElement(age=Age(iso8601duration="P5Y")
                      ))
                      

then the packet

In [13]:
pkt = Phenopacket(
    id="PP1",
    phenotypicFeatures=[pf1],
    metaData=MetaData(
        created="2021-01-01",
    )
)

## Exporting to JSON

Pydantic gives us a simple way to export to JSON

In [17]:
print(pkt.model_dump_json(indent=2))

{
  "biosamples": [],
  "diseases": [],
  "files": [],
  "id": "PP1",
  "interpretations": [],
  "measurements": [],
  "medicalActions": [],
  "metaData": {
    "created": "2021-01-01",
    "createdBy": null,
    "externalReferences": [],
    "phenopacketSchemaVersion": null,
    "resources": [],
    "submittedBy": null,
    "updates": []
  },
  "phenotypicFeatures": [
    {
      "description": null,
      "evidence": [],
      "excluded": null,
      "modifiers": [],
      "onset": {
        "age": {
          "iso8601duration": "P5Y"
        },
        "ageRange": null,
        "gestationalAge": null,
        "interval": null,
        "ontologyClass": null,
        "timestamp": null
      },
      "resolution": null,
      "severity": null,
      "type": {
        "id": "HP:0030084",
        "label": "Clinodactyly"
      }
    }
  ],
  "subject": null
}


## Importing from JSON

Everything works in reverse too

In [24]:
import json
json_str = pkt.model_dump_json(indent=2)
pkt2 = Phenopacket(**json.loads(json_str))
print(pkt2.phenotypicFeatures[0].type.label)

Clinodactyly


## Validation

One advantage of Pydantic is that it gives us validation *at the time of object creation*

(It also provides many type hints in your IDE, which can be very helpful)

Remember, all of this comes from the LinkML schema - we didn't manually author any pydantic.

Let's try and make a feature without an HPO ID:

In [25]:
pf1 = PhenotypicFeature(
    type=OntologyClass(label="Clinodactyly"),
    onset=TimeElement(age=Age(iso8601duration="P5Y")
                      ))


ValidationError: 1 validation error for OntologyClass
id
  Field required [type=missing, input_value={'label': 'Clinodactyly'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.7/v/missing

Perfect! This is exactly what we want.